<a href="https://colab.research.google.com/github/Logesh001/OCR/blob/main/cvsplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np
from PIL import Image #PIL(Python Imaging Library)

This function takes an image path, coordinates (to define the region to crop), and a saved location. It opens an image, crops it using the provided coordinates, and saves the cropped image at the specified location.[link text](https://)

In [3]:
def crop(image_path, coords, saved_location):
    image_obj = Image.open(image_path)
    cropped_image = image_obj.crop(coords)
    cropped_image.save(saved_location)

The code reads an image named "para2.jpg" using OpenCV's cv2.imread.The height and width of the original image are stored as h1 and w1.
    A blank white image with dimensions slightly larger than the original image is created using NumPy (blank_image).
    The original image is copied onto the blank image with an offset of (11, 11) to give it a margin. The result is stored in the img variable.

In [22]:
orig_img = cv2.imread("/content/drive/MyDrive/Goodnotes_test/test/eng_AF_027.jpg")#/content/drive/MyDrive/Goodnotes/train

h1,w1 = orig_img.shape[:2]

blank_image = np.zeros((h1+20,w1+20,3), np.uint8)
blank_image[:,:] = (255,255,255)
x_offset = y_offset = 11
img = blank_image.copy()
img[y_offset:y_offset+h1, x_offset:x_offset+w1] = orig_img.copy()


gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

th1,thresh1 = cv2.threshold(gray,127,255,cv2.THRESH_TOZERO_INV)
print(f"threshold1:{th1}")

threshold1:127.0


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The copied image (img) is converted to grayscale using cv2.cvtColor.A binary inverse threshold is applied to the grayscale image using cv2.threshold. The threshold value is set to 127, and the result is stored in thresh1

Thresholding is a common image processing technique used to **segment an image to different regions** based on their intensity or color of the pixel

In [24]:
th, threshed = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
print(f"threshold:{th}")

threshold:143.0


  The code finds the non-zero points (foreground pixels) in the thresholded image using cv2.findNonZero. The minimum area rectangle is fitted around these points using cv2.minAreaRect, which returns the rectangle's center, size, and angle of rotation.

In [25]:
pts = cv2.findNonZero(threshed)
ret = cv2.minAreaRect(pts)

(cx,cy), (w,h), ang = ret
#if w>h:
 #   w,h = h,w
 #   ang += 90
print(f"ang:{ang}")
ang = 0
th = 10

ang:90.0


  A rotation matrix (M) is generated using cv2.getRotationMatrix2D to rotate the image by a specified angle around the rectangle's center. The thresholded image is rotated using cv2.warpAffine with the calculated rotation matrix (M).
  Histograms are computed along the rows of the rotated and grayscale images using cv2.reduce. These histograms are used to detect upper and lower boundaries of lines in the text.

In [26]:
M = cv2.getRotationMatrix2D((cx,cy), ang, 1.0)
rotated = cv2.warpAffine(threshed, M, (img.shape[1], img.shape[0]))

  The code processes the histograms to identify the upper and lower boundaries of lines of text.
    The upper and lower boundaries are determined by changes in histogram values. If the value transitions from below a threshold (th) to above the threshold or vice versa, it's considered a boundary.
    Detected upper and lower boundaries are stored in the uppers and lowers lists.

  The rotated image is converted back to a BGR image using cv2.cvtColor.
    Lines representing the upper and lower boundaries are drawn on the rotated image using cv2.line.
    For each detected upper and lower boundary pair, the code checks if the distance between them is large enough to indicate a significant block of text. If so, it calculates the coordinates to crop the original image and calls the crop function to save the cropped region.

The rotated and annotated image is saved as "result.png" using cv2.imwrite

In [27]:
hist = cv2.reduce(rotated,1, cv2.REDUCE_AVG).reshape(-1)
hist1 = cv2.reduce(gray,1, cv2.REDUCE_AVG).reshape(-1)

# Create function that adds 100 to something
sub_255 = lambda i: i - 255
# Create vectorized function
vectorized_sub_255 = np.vectorize(sub_255)
#print(f"his1:{vectorized_sub_255(hist1)}")
print(f"hist:{hist}")
print(f"threshold:{th}")
H,W = img.shape[:2]
print(f"H:{H}")
print(f"size:{len(hist)}")
uppers = [y-y_offset for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
lowers = [y-y_offset for y in range(H-1) if hist[y]>th and hist[y+1]<=th]
print(uppers)
print(lowers)

rotated = cv2.cvtColor(rotated, cv2.COLOR_GRAY2BGR)

for y in uppers:
    cv2.line(rotated, (0,y-15), (W, y-15), (255,0,0), 1)

for y in lowers:
    cv2.line(rotated, (0,y+15), (W, y+15), (0,255,0), 1)

i=0
for y in uppers:
    upper_y = y
    if i >= len(lowers):
        continue
    lower_y = lowers[i]
    i=i+1
    print(f"lower:{lower_y} upper:{upper_y}")
    if (lower_y - upper_y) < 10:
        continue
    coords = [0,upper_y-15,W,lower_y+15]
    crop("/content/drive/MyDrive/Goodnotes_test/test/eng_AF_027.jpg",coords,f"save{i}.png")

cv2.imwrite("result.png", rotated)


hist:[0 0 0 ... 0 0 0]
threshold:10
H:1726
size:1726
[-1, 190, 222, 224, 228, 273, 280, 283, 285, 288, 296, 343, 347, 350, 366, 422, 427, 436, 483, 519, 533, 552, 578, 593, 605, 626, 679, 686, 695, 743]
[183, 221, 223, 226, 236, 276, 282, 284, 287, 289, 302, 346, 348, 351, 371, 423, 430, 441, 517, 524, 551, 573, 583, 597, 606, 661, 684, 689, 737, 1705]
lower:183 upper:-1
lower:221 upper:190
lower:223 upper:222
lower:226 upper:224
lower:236 upper:228
lower:276 upper:273
lower:282 upper:280
lower:284 upper:283
lower:287 upper:285
lower:289 upper:288
lower:302 upper:296
lower:346 upper:343
lower:348 upper:347
lower:351 upper:350
lower:371 upper:366
lower:423 upper:422
lower:430 upper:427
lower:441 upper:436
lower:517 upper:483
lower:524 upper:519
lower:551 upper:533
lower:573 upper:552
lower:583 upper:578
lower:597 upper:593
lower:606 upper:605
lower:661 upper:626
lower:684 upper:679
lower:689 upper:686
lower:737 upper:695
lower:1705 upper:743


True